In [4]:
# 基于Module类的模型构造方法，让模型构造更加灵活
import torch
from torch import nn

class MLP(nn.Module):
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256,10)
        
    def forward(self,x):
        a = self.act(self.hidden(x))
        return self.output(a)
    
x = torch.rand(2,784)
net = MLP()
print(net)
net(x)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0639,  0.1164,  0.0348, -0.0881,  0.0459,  0.1161,  0.1071, -0.2703,
         -0.0498, -0.0666],
        [-0.0434,  0.0906, -0.0567, -0.1475, -0.0518, -0.0238,  0.0236, -0.2111,
         -0.1439, -0.1320]], grad_fn=<AddmmBackward>)

In [12]:
# 基于Module类的模型构造方法，让模型构造更加灵活
import torch
from torch import nn

# Module类的子类 Sequential、ModuleList、ModuleDict等
# Sequrntial 定义一个MySequential类来了解工作机制
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super(MySequential,self).__init__()
        if len(args) == 1 and isinstance(args[0],OrderedDict):
            for key, module in args[0].items():
                self.add_module(key,module)
        else:
            for idx,module in enumerate(args):
                self.add_module(str(idx),module)
                    
    def forward(self,input):
        for module in self._modules.values():
            input = module(input)
        return input

net = MySequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10),
)

print(net)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [16]:
# 基于Module类的模型构造方法，让模型构造更加灵活
import torch
from torch import nn

# ModuleList类
net = nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net.append(nn.Linear(256,10))
# print(net[-1])
# print(net)

# ModuleList与Sequential的区别：ModuleList中的模块的连接顺序没有确定，而Sequential
# 中模块间的前后顺序是确定的，并且Sequential中已经自带forward（前向计算）功能，ModuleList
# 中没有

class MyModule(nn.Module):
    def __init__(self):
        super(MyModule,self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10,10) for i in range(10)])
        
    def forward(self,x):
        for i,l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x
    
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList,self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10,10)])

class Module_List(nn.Module):
    def __init__(self):
        super(Module_List,self).__init__()
        self.linears = [nn.Linear(10,10)]
        
    
net1 = Module_ModuleList()
net2 = Module_List()

print("net1")
for p in net1.parameters():
    print(p.size())
print("net2")
for p in net2.parameters():
    print(p.size())




net1
torch.Size([10, 10])
torch.Size([10])
net2


In [17]:
# 基于Module类的模型构造方法，让模型构造更加灵活
import torch
from torch import nn
# ModuleDict类
# Module接收一个子模块的字典作为输入，然后也可以类似字典那样进行访问和添加操作
net = nn.ModuleDict({
    'linear':nn.Linear(784,256),
    'act':nn.ReLU(),
})

net['output'] = nn.Linear(256,10)
print(net['linear'])
print(net.output)
print(net)




Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [18]:
# 基于Module类的模型构造方法，让模型构造更加灵活
import torch
from torch import nn
# 构造复杂的模型

class FancyMLP(nn.Module):
    def __init__(self):
        super(FancyMLP,self).__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)
        
    def forward(self,x):
        x = self.linear(x)
        x = nn.functional.relu(torch.mm(x,self.rand_weight) + 1)
        
        x = self.linear(x)# 复用全连接层，相当于两个全连接层共享参数
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()
    
X = torch.rand(2,20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(16.0080, grad_fn=<SumBackward0>)

In [20]:
# 基于Module类的模型构造方法，让模型构造更加灵活
import torch
from torch import nn
# Fancy和Sequential都是Module的子类，我们可以嵌套调用他们

# class FancyMLP(nn.Module):
#     def __init__(self):
#         super(FancyMLP,self).__init__()
#         self.rand_weight = torch.rand((20,20),requires_grad=False)
#         self.linear = nn.Linear(20,20)
        
#     def forward(self,x):
#         x = self.linear(x)
#         x = nn.functional.relu(torch.mm(x,self.rand_weight) + 1)
        
#         x = self.linear(x)# 复用全连接层，相当于两个全连接层共享参数
#         while x.norm().item() > 1:
#             x /= 2
#         if x.norm().item() < 0.8:
#             x *= 10
#         return x.sum()
    
class NestMLP(nn.Module):
    def __init__(self,**kargs):
        super(NestMLP,self).__init__(**kargs)
        self.net = nn.Sequential(nn.Linear(40,30),nn.ReLU())
        
    def forward(self,x):
        return self.net(x)

net = nn.Sequential(NestMLP(),nn.Linear(30,20),FancyMLP())

x = torch.rand(2,40)
print(net)
net(x)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(2.1981, grad_fn=<SumBackward0>)